In [ ]:
import h5py
import os

std_path = "../DATASET/RAW/STD"
files = os.listdir(std_path)

print("Total STD files:", len(files))
print("Sample file:", files[0])

file_path = os.path.join(std_path, files[0])

with h5py.File(file_path, "r") as f:
    print("Keys in STD file:")
    for key in f.keys():
        print(key)


In [ ]:
hem_path = "../DATASET/RAW/HEM"
files = os.listdir(hem_path)

print("Total HEM files:", len(files))
print("Sample file:", files[0])

file_path = os.path.join(hem_path, files[0])

with h5py.File(file_path, "r") as f:
    print("Keys in HEM file:")
    for key in f.keys():
        print(key)


In [ ]:
import h5py
import pickle
from pathlib import Path

# --------------------------------------------------
# 1️⃣ Define paths relative to NOTEBOOKS folder
# --------------------------------------------------
PROJECT_ROOT = Path("..").resolve()

STD_DIR = PROJECT_ROOT / "DATASET" / "RAW" / "STD"
HEM_DIR = PROJECT_ROOT / "DATASET" / "RAW" / "HEM"
MATCHED_FILE = PROJECT_ROOT / "DATASET" / "PROCESSED" / "matched_files.pkl"

print("Project Root:", PROJECT_ROOT)
print("STD Path:", STD_DIR)
print("HEM Path:", HEM_DIR)

# --------------------------------------------------
# 2️⃣ Load matched file list
# --------------------------------------------------
with open(MATCHED_FILE, "rb") as f:
    matched_data = pickle.load(f)

print("Total matched samples:", len(matched_data))

# Take first sample
sample = matched_data[0]

std_path = STD_DIR / sample["std_file"]
hem_path = HEM_DIR / sample["hem_file"]

print("STD file:", std_path)
print("HEM file:", hem_path)

# --------------------------------------------------
# 3️⃣ Inspect STD file
# --------------------------------------------------
with h5py.File(std_path, "r") as f:
    print("\nSTD Keys:")
    print(list(f.keys()))

# --------------------------------------------------
# 4️⃣ Inspect HEM file
# --------------------------------------------------
with h5py.File(hem_path, "r") as f:
    print("\nHEM Keys:")
    print(list(f.keys()))


In [ ]:
import numpy as np
import h5py

with h5py.File(std_path, "r") as f:
    tir1 = f["TIR1_BT"][:]
    tir2 = f["TIR2_BT"][:]
    wv   = f["WV_BT"][:]

print("TIR1 shape:", tir1.shape, "min:", np.min(tir1), "max:", np.max(tir1))
print("TIR2 shape:", tir2.shape, "min:", np.min(tir2), "max:", np.max(tir2))
print("WV shape:", wv.shape, "min:", np.min(wv), "max:", np.max(wv))

with h5py.File(hem_path, "r") as f:
    hem = f["HEM"][:]

print("HEM shape:", hem.shape, "min:", np.min(hem), "max:", np.max(hem))


In [ ]:
import numpy as np
import cv2

# Remove extra dimension
tir1 = tir1[0]
tir2 = tir2[0]
wv   = wv[0]
hem  = hem[0]

print("After squeeze:")
print("TIR1:", tir1.shape)
print("WV:", wv.shape)
print("HEM:", hem.shape)


In [ ]:
wv_resized = cv2.resize(
    wv,
    (tir1.shape[1], tir1.shape[0]),
    interpolation=cv2.INTER_LINEAR
)

print("WV resized shape:", wv_resized.shape)


In [ ]:
def normalize_bt(x):
    return (x - 180) / (320 - 180)

tir1 = normalize_bt(tir1)
tir2 = normalize_bt(tir2)
wv_resized = normalize_bt(wv_resized)


In [ ]:
target_size = (512, 512)

X_resized = np.stack([
    cv2.resize(X[i], target_size, interpolation=cv2.INTER_LINEAR)
    for i in range(3)
])

Y_resized = cv2.resize(Y, target_size, interpolation=cv2.INTER_NEAREST)

print("Downsampled X:", X_resized.shape)
print("Downsampled Y:", Y_resized.shape)


In [ ]:
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()

Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_event.npy")

Y_t = Y[:-1]
Y_t1 = Y[1:]

same = np.sum(Y_t == Y_t1)
total = len(Y_t)

print("Persistence rate:", same / total)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

from pathlib import Path
PROJECT_ROOT = Path("..").resolve()

Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_event.npy")

Y_t = Y[:-1]
Y_t1 = Y[1:]

acc = accuracy_score(Y_t1, Y_t)
f1 = f1_score(Y_t1, Y_t)

print("Persistence Accuracy:", acc)
print("Persistence F1:", f1)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()

Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_event.npy")

lead = 3
Y_t = Y[:-lead]
Y_t3 = Y[lead:]

acc = accuracy_score(Y_t3, Y_t)
f1 = f1_score(Y_t3, Y_t)

print("Persistence t+3 Accuracy:", acc)
print("Persistence t+3 F1:", f1)



In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()

Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_event.npy")

lead = 6

Y_t = Y[:-lead]
Y_t6 = Y[lead:]

acc = accuracy_score(Y_t6, Y_t)
f1 = f1_score(Y_t6, Y_t)

print("Persistence t+6 Accuracy:", acc)
print("Persistence t+6 F1:", f1)



In [ ]:
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()

Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_event.npy")

history = 3
lead = 6

start_idx = history - 1
end_idx = len(Y) - lead

Y_t6 = Y[start_idx+lead : end_idx+lead]

print("Validation rain ratio:",
      np.mean(Y_t6[int(0.7*len(Y_t6)) : int(0.85*len(Y_t6))]))


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()

Y_onset = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_onset_t3.npy")

Y_pred = np.zeros_like(Y_onset)

acc = accuracy_score(Y_onset, Y_pred)
f1 = f1_score(Y_onset, Y_pred)

print("Persistence (always no-onset)")
print("Accuracy:", acc)
print("F1:", f1)


In [ ]:
import numpy as np
Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_onset_t3.npy")
start = 3-1
n = len(Y)
train_end = int(0.7*n); val_end=int(0.85*n)
print(np.mean(Y[:train_end]), np.mean(Y[train_end:val_end]), np.mean(Y[val_end:]))



In [ ]:
import pickle
from pathlib import Path

# Manually set project root
PROJECT_ROOT = Path("/Users/apple/Documents/ElteBook/Code/PROJECT/XAI NEW")

with open(PROJECT_ROOT / "DATASET/PROCESSED/matched_files.pkl", "rb") as f:
    matched_files = pickle.load(f)

print("Type of matched_files:", type(matched_files))
print("Type of first element:", type(matched_files[0]))
print("First element:")
print(matched_files[0])


In [ ]:
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path("/Users/apple/Documents/ElteBook/Code/PROJECT/XAI NEW")

HEM = np.load(PROJECT_ROOT / "DATASET/PROCESSED/HEM_256.npy")

print("Min:", HEM.min())
print("Max:", HEM.max())
print("Mean:", HEM.mean())
print("Percentiles:", np.percentile(HEM, [50, 75, 90, 95, 99]))


In [ ]:
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path("/Users/apple/Documents/ElteBook/Code/PROJECT/XAI NEW")

HEM = np.load(PROJECT_ROOT / "DATASET/PROCESSED/HEM_256.npy")

thresholds = [0.2, 0.5, 1, 2, 5]

for t in thresholds:
    frame_positive = []
    for frame in HEM:
        coverage = np.mean(frame > t)
        frame_positive.append(coverage > 0.005)

    print(f"Threshold {t} → Event ratio: {np.mean(frame_positive)}")


In [ ]:
import numpy as np
from pathlib import Path

BASE = Path("/Users/apple/Documents/ElteBook/Code/PROJECT/XAI NEW/DATASET/PROCESSED")

X = np.load(BASE / "X_patch_event.npy")
Y = np.load(BASE / "Y_patch_event.npy")

print("X shape:", X.shape)
print("Y shape:", Y.shape)
print("Event ratio:", Y.mean())


In [ ]:
import numpy as np
from pathlib import Path
from PIL import Image

BASE = Path("/Users/apple/Documents/ElteBook/Code/PROJECT/XAI NEW/DATASET/PROCESSED")

X = np.load(BASE / "X_patch_event.npy")

# choose one patch
sample = X[100]  # change index if needed

# convert to HWC format
sample = np.transpose(sample, (1,2,0))

# normalize to 0-255 for saving
sample = (sample - sample.min()) / (sample.max() - sample.min())
sample = (sample * 255).astype(np.uint8)

img = Image.fromarray(sample)
img.save("demo_patch.png")

print("Saved demo_patch.png")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

# 🔹 Set your project root manually
PROJECT_ROOT = Path("/Users/apple/Documents/ElteBook/Code/PROJECT/XAI NEW")

# 🔹 Load real patch dataset (the one used for training)
X_patch = np.load(PROJECT_ROOT / "DATASET/PROCESSED/X_patch_event.npy")
Y_patch = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_patch_event.npy")

print("Dataset shape:", X_patch.shape)
print("Event ratio:", Y_patch.mean())

# 🔹 Pick a TRUE POSITIVE patch
positive_indices = np.where(Y_patch == 1)[0]
idx = positive_indices[0]

patch = X_patch[idx]  # (3, 64, 64)

print("Selected index:", idx)
print("Patch shape:", patch.shape)

# 🔹 Convert to displayable image (for frontend demo only)
patch_vis = np.transpose(patch, (1, 2, 0))  # (64,64,3)

# Normalize for visualization
patch_vis = patch_vis - patch_vis.min()
patch_vis = patch_vis / (patch_vis.max() + 1e-6)

patch_vis_uint8 = (patch_vis * 255).astype(np.uint8)

# 🔹 Save PNG for frontend testing
save_path = PROJECT_ROOT / "NOTEBOOK/demo_patch.png"
Image.fromarray(patch_vis_uint8).save(save_path)

print("✅ Demo patch saved at:", save_path)

# 🔹 Show inline
plt.imshow(patch_vis_uint8)
plt.title("Real Training Patch")
plt.axis("off")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

# Update this path if needed
PROJECT_ROOT = Path("/Users/apple/Documents/ElteBook/Code/PROJECT/XAI NEW")

# Load dataset
X = np.load(PROJECT_ROOT / "DATASET/PROCESSED/X_patch_event.npy")
Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_patch_event.npy")

print("Dataset shape:", X.shape)
print("Event ratio:", Y.mean())

# 🔥 Find first heavy rain patch
heavy_indices = np.where(Y == 1)[0]
idx = heavy_indices[0]

print("Selected heavy index:", idx)

patch = X[idx]  # shape (3, 64, 64)

# Convert to displayable image
patch_img = np.transpose(patch, (1, 2, 0))  # CHW -> HWC

# Normalize for visualization
patch_img = (patch_img - patch_img.min()) / (patch_img.max() - patch_img.min() + 1e-8)

# Save PNG
save_path = PROJECT_ROOT / "NOTEBOOKs/heavy_demo.png"
save_path.parent.mkdir(parents=True, exist_ok=True)

Image.fromarray((patch_img * 255).astype(np.uint8)).save(save_path)

print("✅ Heavy rain patch saved at:", save_path)

# Show it
plt.imshow(patch_img)
plt.title("Heavy Rain Patch")
plt.axis("off")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import random

PROJECT_ROOT = Path("/Users/apple/Documents/ElteBook/Code/PROJECT/XAI NEW")

X = np.load(PROJECT_ROOT / "DATASET/PROCESSED/X_patch_event.npy")
Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_patch_event.npy")

heavy_indices = np.where(Y == 1)[0]

print("Total heavy patches:", len(heavy_indices))

# 🔥 Random heavy patch
idx = random.choice(heavy_indices)

patch = X[idx]

img = patch.transpose(1,2,0)

# Normalize for saving
img_norm = img - img.min()
img_norm = img_norm / (img_norm.max() + 1e-8)

img_uint8 = (img_norm * 255).astype(np.uint8)

filename = f"heavy_demo_{idx}.png"
plt.imsave(filename, img_uint8)

plt.imshow(img_uint8)
plt.title(f"Heavy Patch Index {idx}")
plt.axis("off")

print("Saved:", filename)


In [ ]:
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent

X = np.load(PROJECT_ROOT / "DATASET/PROCESSED/X_patch_multiframe_t3.npy")
Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_patch_multiframe_t3.npy")

idx = np.where(Y == 1)[0][0]

sample = X[idx]

np.save(PROJECT_ROOT / "NOTEBOOKs/heavy_sample.npy", sample)

print("Saved heavy_sample.npy at:", PROJECT_ROOT)


In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
PROJECT_ROOT = Path().resolve().parent

X = np.load(PROJECT_ROOT / "DATASET/PROCESSED/X_patch_multiframe_t3.npy")
Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_patch_multiframe_t3.npy")

print("Dataset shape:", X.shape)
print("Event ratio:", Y.mean())


Dataset shape: (49104, 9, 64, 64)
Event ratio: 0.22896301726946888


In [ ]:
def save_samples(class_value, count=3):
    indices = np.where(Y == class_value)[0]
    np.random.shuffle(indices)

    selected = indices[:count]

    for i, idx in enumerate(selected):
        sample = X[idx]

        # Save .npy file
        np.save(PROJECT_ROOT / f"sample_{class_value}_{i}.npy", sample)

        # Save PNG preview (first 3 channels only)
        img = sample[:3]  # first time frame
        img = np.transpose(img, (1,2,0))

        # normalize for visualization
        img = (img - img.min()) / (img.max() - img.min() + 1e-8)

    print(f"Saved {count} samples for class {class_value}")


In [3]:
# Heavy Rain samples
save_samples(class_value=1, count=3)

# No Heavy Rain samples
save_samples(class_value=0, count=3)


Saved 3 samples for class 1
Saved 3 samples for class 0


In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

PROJECT_ROOT = Path().resolve().parent

X = np.load(PROJECT_ROOT / "DATASET/PROCESSED/X_patch_multiframe_t3.npy")
Y = np.load(PROJECT_ROOT / "DATASET/PROCESSED/Y_patch_multiframe_t3.npy")

print("Dataset shape:", X.shape)
print("Event ratio:", Y.mean())


def save_samples(class_value, count=5):
    indices = np.where(Y == class_value)[0]
    
    if len(indices) < count:
        print(f"Not enough samples for class {class_value}")
        return
    
    np.random.shuffle(indices)
    selected = indices[:count]

    for i, idx in enumerate(selected):
        sample = X[idx]

        # 🔹 Save .npy file
        np.save(PROJECT_ROOT / f"sample_class{class_value}_{i}.npy", sample)

        # 🔹 Save PNG preview (first time frame only)
        img = sample[:3]  # first time step (3 channels)
        img = np.transpose(img, (1, 2, 0))

        # normalize for visualization
        img = (img - img.min()) / (img.max() - img.min() + 1e-8)

        plt.imsave(PROJECT_ROOT / f"sample_class{class_value}_{i}.png", img)

    print(f"Saved {count} samples for class {class_value}")


# ✅ 5 Heavy Rain samples
save_samples(class_value=1, count=5)

# ✅ 5 No Heavy Rain samples
save_samples(class_value=0, count=5)

print("✅ Total 10 samples saved successfully.")
